# Демонстрационный практикум: Алгоритм Форда-Фалкерсона
## Максимальный поток в газотранспортной сети
Тема: Анализ алгоритма Форда-Фалкерсона на примере оптимизации транспортировки газа через российскую газотранспортную систему

### 1. Введение

### 1.1 Теоретические основы

Алгоритм Форда-Фалкерсона — это классический алгоритм для решения задачи о максимальном потоке в сетевых графах. Задача максимального потока находит применение в:
Оптимизации транспортировки нефти, газа и других ресурсов
Проектировании телекоммуникационных сетей
Управлении логистическими системами
Анализе пропускной способности инфраструктуры

### 1.2 Описание задачи

Задача о максимальном потоке: Дан ориентированный граф G=(V,E), где каждому ребру (u,v) присвоена пропускная способность c(u,v)>0. Необходимо найти максимальный поток f из источника s в сток t, такой что:
Ограничение пропускной способности: f(u,v)≤c(u,v) для каждого ребра
Сохранение потока (кроме источника и стока): u f(u,v)=w f(v,w)

### 1.3 Основная идея алгоритма

Алгоритм Форда-Фалкерсона работает итеративно:
Инициализация: начинаем с нулевого потока по всем рёбрам
Поиск увеличивающего пути: находим путь от источника к стоку в остаточном графе
Увеличение потока: увеличиваем поток по найденному пути на минимальную остаточную пропускную способность
Повторение: повторяем шаги 2-3 до тех пор, пока существуют увеличивающие пути

## 2. Практическое приложение: Газотранспортная система России

### 2.1 Контекст задачи

Российская газотранспортная система (ГТС) — одна из крупнейших в мире. Она состоит из:
Источников производства: газовые месторождения в Западной Сибири, на Сахалине, Кавказе
Компрессорных станций: повышают давление для транспортировки на дальние расстояния
Магистральных газопроводов: протянуты на десятки тысяч километров
Потребителей: европейские страны, азиатские государства, отечественные потребители

### 2.2 Модель как задача максимального потока

Газотранспортную систему можно представить как направленный граф, где:
Вершины = станции (источники, компрессорные станции, потребители)
Рёбра = газопроводы между станциями
Пропускная способность рёбер = пропускная способность трубопровода (млрд м³/год)
Источник = основные месторождения
Сток = суммарный спрос потребителей
Цель: найти максимальный объём газа, который можно транспортировать из источников к потребителям при учёте пропускных способностей каждого газопровода.

## 3. Реализация на Python 3

### 3.1 Полная реализация алгоритма Форда-Фалкерсона

from collections import defaultdict, deque
from typing import Dict, List, Tuple
class GasTransportNetwork:

Класс для моделирования газотранспортной системы
и решения задачи о максимальном потоке.



In [ ]:
from collections import defaultdict, deque
from typing import Dict, List, Tuple

class GasTransportNetwork:
    """Класс для моделирования газотранспортной системы
    и решения задачи о максимальном потоке."""

    def __init__(self):
        # Остаточный граф представлен в виде матрицы смежности
        # graph[u][v] = пропускная способность от узла u к узлу v
        self.graph = defaultdict(lambda: defaultdict(int))
        # Словарь для хранения найденного потока
        self.flow = defaultdict(lambda: defaultdict(int))
        # Список всех узлов в сети
        self.nodes = set()

    def add_edge(self, source: str, sink: str, capacity: int):
        """
        Добавляет ребро в граф (газопровод между двумя станциями).

        Args:
            source (str): исходный узел (название станции)
            sink (str): целевой узел (название станции)
            capacity (int): пропускная способность газопровода
        """
        # Добавляем рёбра в оба направления (для остаточного графа)
        self.graph[source][sink] += capacity
        # Обратное ребро инициализируется нулём
        if sink not in self.graph or source not in self.graph[sink]:
            self.graph[sink][source] = 0
        # Добавляем узлы в множество
        self.nodes.add(source)
        self.nodes.add(sink)

    def bfs(self, source: str, sink: str, parent: Dict) -> bool:
        """
        Поиск увеличивающего пути методом поиска в ширину (BFS).
        Это реализация алгоритма Edmonds-Karp (улучшение Форда-Фалкерсона).

        Args:
            source (str): узел-источник
            sink (str): узел-сток
            parent (Dict): словарь для восстановления пути

        Returns:
            bool: True если путь найден, False иначе
        """
        # Множество посещённых узлов
        visited = set()
        # Очередь для BFS
        queue = deque([source])
        visited.add(source)

        while queue:
            # Извлекаем узел из начала очереди
            u = queue.popleft()

            # Проверяем всех соседей текущего узла
            for v in self.graph[u]:
                # Если сосед не посещён и есть остаточная пропускная способность
                if v not in visited and self.graph[u][v] > 0:
                    visited.add(v)
                    queue.append(v)
                    # Сохраняем родителя для восстановления пути
                    parent[v] = u
                    # Если достигли стока, путь найден
                    if v == sink:
                        return True

        return False

    def ford_fulkerson(self, source: str, sink: str) -> int:
        """
        Алгоритм Форда-Фалкерсона (с BFS = Edmonds-Karp).
        Находит максимальный поток от source к sink.

        Args:
            source (str): узел-источник (главное месторождение газа)
            sink (str): узел-сток (главный потребитель)

        Returns:
            int: максимальный поток (млрд м³/год)
        """
        # Инициализируем максимальный поток нулём
        max_flow_value = 0
        # Словарь для восстановления пути из BFS
        parent = {}

        # Основной цикл: повторяем, пока существуют увеличивающие пути
        while self.bfs(source, sink, parent):
            # Находим минимальную пропускную способность на пути
            # (узкое место маршрута)
            path_flow = float('inf')
            s = sink

            # Идём по пути от стока к источнику
            while s != source:
                u = parent[s]
                # Берём минимум из пропускных способностей
                path_flow = min(path_flow, self.graph[u][s])
                s = u

            # Добавляем найденный поток к общему потоку
            max_flow_value += path_flow

            # Обновляем остаточные пропускные способности
            # (основная идея: уменьшаем пропускную способность прямого ребра,
            # увеличиваем пропускную способность обратного ребра)
            v = sink
            while v != source:
                u = parent[v]
                # Уменьшаем пропускную способность от u к v
                self.graph[u][v] -= path_flow
                # Увеличиваем пропускную способность от v к u (обратное ребро)
                self.graph[v][u] += path_flow
                # Сохраняем информацию о потоке для последующего анализа
                self.flow[u][v] += path_flow
                v = u

            # Очищаем словарь родителей для следующей итерации
            parent = {}

        return max_flow_value

    def get_flow_on_edge(self, u: str, v: str) -> int:
        """
        Возвращает поток по конкретному ребру (газопроводу).

        Args:
            u (str): исходный узел
            v (str): целевой узел

        Returns:
            int: величина потока по ребру
        """
        return self.flow[u][v]

    def print_flow_solution(self, source: str, sink: str, max_flow_value: int):
        """
        Выводит решение в красивом формате с анализом каждого газопровода.

        Args:
            source (str): узел-источник
            sink (str): узел-сток
            max_flow_value (int): значение максимального потока
        """
        print("\n" + "="*70)
        print("РЕШЕНИЕ: МАКСИМАЛЬНЫЙ ПОТОК В ГАЗОТРАНСПОРТНОЙ СЕТИ")
        print("="*70)
        print(f"\nМаксимальный поток от '{source}' к '{sink}': {max_flow_value} млрд м³/год")
        print("\nПоток по каждому газопроводу:")
        print("-"*70)

        # Выводим поток по каждому используемому газопроводу
        for u in sorted(self.flow.keys()):
            for v in sorted(self.flow[u].keys()):
                if self.flow[u][v] > 0:
                    print(f"  {u:20} → {v:20} : {self.flow[u][v]:3} млрд м³/год")

        print("-"*70)


### 3.2 Применение к моделированию российской ГТС


In [ ]:
def demonstrate_russian_gas_network():
   """Демонстрация работы алгоритма на примере упрощённой модели
   российской газотранспортной системы."""


In [ ]:
print("\n" + "="*70)
print("АЛГОРИТМ ФОРДА-ФАЛКЕРСОНА: МАКСИМАЛЬНЫЙ ПОТОК ГАЗ")
print("Применение к газотранспортной системе России")
print("="*70)

# Создаём объект сети
network = GasTransportNetwork()

# Построение упрощённой модели газотранспортной системы
# (на основе реальной топологии)

print("\n1. ПОСТРОЕНИЕ МОДЕЛИ СЕТИ")
print("-"*70)
print("Добавляем станции (вершины) и газопроводы (рёбра) с пропускными способностями:\n")

# Основные месторождения (источники)
# Добавляем рёбра (газопровод): (источник, назначение, пропускная способность)

# От месторождений Западной Сибири к компрессорным станциям
network.add_edge("Месторождение_Уренгой", "КС_Муравленко", 180)
print("  Месторождение_Уренгой → КС_Муравленко: 180 млрд м³/год")

network.add_edge("Месторождение_Уренгой", "КС_Новый_Уренгой", 150)
print("  Месторождение_Уренгой → КС_Новый_Уренгой: 150 млрд м³/год")

network.add_edge("Месторождение_Ямбург", "КС_Новый_Уренгой", 120)
print("  Месторождение_Ямбург → КС_Новый_Уренгой: 120 млрд м³/год")

# От компрессорных станций к распределительным узлам
network.add_edge("КС_Муравленко", "Магистраль_Центральная", 160)
print("  КС_Муравленко → Магистраль_Центральная: 160 млрд м³/год")

network.add_edge("КС_Новый_Уренгой", "Магистраль_Центральная", 190)
print("  КС_Новый_Уренгой → Магистраль_Центральная: 190 млрд м³/год")

network.add_edge("КС_Новый_Уренгой", "Магистраль_Альтернативная", 100)
print("  КС_Новый_Уренгой → Магистраль_Альтернативная: 100 млрд м³/год")

# От магистралей к потребителям (граница с Европой, Азия)
network.add_edge("Магистраль_Центральная", "Потребители_Европа", 200)
print("  Магистраль_Центральная → Потребители_Европа: 200 млрд м³/год")

network.add_edge("Магистраль_Центральная", "Потребители_Азия", 150)
print("  Магистраль_Центральная → Потребители_Азия: 150 млрд м³/год")

network.add_edge("Магистраль_Альтернативная", "Потребители_Азия", 120)
print("  Магистраль_Альтернативная → Потребители_Азия: 120 млрд м³/год")

# Виртуальные рёбра от потребителей к общему стоку
network.add_edge("Потребители_Европа", "СТОК", 200)
print("  Потребители_Европа → СТОК: 200 млрд м³/год")

network.add_edge("Потребители_Азия", "СТОК", 270)
print("  Потребители_Азия → СТОК: 270 млрд м³/год")

# Виртуальное ребро от общего источника к месторождениям
network.add_edge("ИСТОЧНИК", "Месторождение_Уренгой", 330)
print("  ИСТОЧНИК → Месторождение_Уренгой: 330 млрд м³/год")

network.add_edge("ИСТОЧНИК", "Месторождение_Ямбург", 120)
print("  ИСТОЧНИК → Месторождение_Ямбург: 120 млрд м³/год")

# Решаем задачу о максимальном потоке
print("\n2. РЕШЕНИЕ ЗАДАЧИ ФОРДА-ФАЛКЕРСОНА")
print("-"*70)

source_node = "ИСТОЧНИК"
sink_node = "СТОК"

# Вычисляем максимальный поток
max_flow_result = network.ford_fulkerson(source_node, sink_node)

# Выводим результат
network.print_flow_solution(source_node, sink_node, max_flow_result)

# Анализ результатов
print("\n3. АНАЛИЗ И ИНТЕРПРЕТАЦИЯ РЕЗУЛЬТАТОВ")
print("-"*70)
print(f"""Максимальный поток газа через сеть составляет {max_flow_result} млрд м³/год. \nЭто означает, что при текущей конфигурации и пропускных способностях газопроводов, \nроссийская газотранспортная система может максимально транспортировать \nуказанный объём газа от источников к потребителям. \n\nДальнейший анализ потоков по отдельным рёбрам (газопроводам), \nпредставленный выше, показывает, какие участки сети используются наиболее интенсивно \nи где могут быть 'бутылочные горлышки' (узкие места), ограничивающие общий поток.""")



АЛГОРИТМ ФОРДА-ФАЛКЕРСОНА: МАКСИМАЛЬНЫЙ ПОТОК ГАЗ
Применение к газотранспортной системе России

1. ПОСТРОЕНИЕ МОДЕЛИ СЕТИ
----------------------------------------------------------------------
Добавляем станции (вершины) и газопроводы (рёбра) с пропускными способностями:

  Месторождение_Уренгой → КС_Муравленко: 180 млрд м³/год
  Месторождение_Уренгой → КС_Новый_Уренгой: 150 млрд м³/год
  Месторождение_Ямбург → КС_Новый_Уренгой: 120 млрд м³/год
  КС_Муравленко → Магистраль_Центральная: 160 млрд м³/год
  КС_Новый_Уренгой → Магистраль_Центральная: 190 млрд м³/год
  КС_Новый_Уренгой → Магистраль_Альтернативная: 100 млрд м³/год
  Магистраль_Центральная → Потребители_Европа: 200 млрд м³/год
  Магистраль_Центральная → Потребители_Азия: 150 млрд м³/год
  Магистраль_Альтернативная → Потребители_Азия: 120 млрд м³/год
  Потребители_Европа → СТОК: 200 млрд м³/год
  Потребители_Азия → СТОК: 270 млрд м³/год
  ИСТОЧНИК → Месторождение_Уренгой: 330 млрд м³/год
  ИСТОЧНИК → Месторождение_Ямбург: 120 

## ВЫВОД ИЗ АЛГОРИТМА:

• Максимальный объём газа, который можно транспортировать: {max_flow_result} млрд м³/год

• Этот результат представляет собой минимальный разрез (min-cut) в сети
по теореме о максимальном потоке и минимальном разрезе (Max-Flow Min-Cut).

• Основные "узкие места" в системе:

Пропускная способность от месторождений к компрессорным станциям

Пропускная способность магистральных линий

Пропускная способность к потребителям

• Практическое применение:

✓ Оптимизация маршрутов транспортировки газа

✓ Определение критических участков сети

✓ Планирование развития газотранспортной инфраструктуры

✓ Прогноз максимальных объёмов экспорта

## Запускаем демонстрацию


In [ ]:
demonstrate_russian_gas_network()

# 4. Сложность и анализ алгоритма
## 4.1 Временная сложность
Реализация
Метод поиска пути
Временная сложность
Примечание
Форд-Фалкерсон (базовый)
DFS
O(E⋅‖f*‖)
Зависит от значения макс. потока
Edmonds-Karp
BFS
O(VE2)
Полиномиальное, не зависит от пропускных способностей
Диниц
DFS с уровнями
O(V2E)
Более эффективно на практике


где:
V = количество вершин (станций)
E = количество рёбер (газопроводов)
‖f*‖ = значение максимального потока

## 4.2 Пространственная сложность

Остаточный граф: O(V2) (матрица смежности) или O(V+E) (список смежности)
Словарь потоков: O(E)
BFS очередь: O(V)
Итого: O(V2) для матрицы смежности, O(V+E) для списков смежности

## 4.3 Практический анализ для газотранспортной системы

Для российской ГТС:
V ≈ 500-1000 (компрессорные станции, узлы распределения)
E ≈ 1500-3000 (газопроводы)
Максимальный поток: 400-500 млрд м³/год
Ожидаемое время выполнения (Edmonds-Karp): < 1 сек на современном ПК

# 5. Дополнительные примеры и вариации

## 5.1 Граф с ограничениями на вершины


In [ ]:
class GasNetworkWithNodeCapacity(GasTransportNetwork):
  """Расширенная версия: каждая станция имеет максимальную пропускную
  способность (потери, техническое обслуживание)."""


In [ ]:
def __init__(self):
    super().__init__()
    # Ограничения пропускной способности вершин
    self.node_capacity = defaultdict(lambda: float('inf'))

def set_node_capacity(self, node: str, capacity: int):
    """Устанавливает максимальную пропускную способность вершины."""
    self.node_capacity[node] = capacity

def add_edge_with_node_constraints(self, source: str, sink: str, capacity: int):
    """
    Добавляет ребро, учитывая ограничения на вершины.
    Решение: разбить вершину на две с ограничивающим ребром.
    """
    # Разбиваем вершину sink на: sink_in и sink_out
    sink_in = f"{sink}_in"
    sink_out = f"{sink}_out"

    # Добавляем основное ребро от source к sink_in
    self.add_edge(source, sink_in, capacity)

    # Добавляем ограничивающее ребро с ёмкостью узла
    self.add_edge(sink_in, sink_out, self.node_capacity[sink])




## 5.2 Граф с затратами на поток

In [ ]:
class CostFlowNetwork:
   """Расширение для учёта стоимости транспортировки газа:
   не только максимальный поток, но и минимальная стоимость."""


In [ ]:
def __init__(self):
    self.graph = defaultdict(lambda: defaultdict(int))
    self.cost = defaultdict(lambda: defaultdict(int))
    self.flow = defaultdict(lambda: defaultdict(int))

def add_edge_with_cost(self, u: str, v: str, capacity: int, cost: float):
    """
    Добавляет ребро с учётом стоимости.

    Args:
        cost: стоимость единицы потока (руб./млрд м³)
    """
    self.graph[u][v] += capacity
    self.graph[v][u] = 0
    self.cost[u][v] = cost
    self.cost[v][u] = -cost


## 6. Ссылки на репозитории

### 6.1 GitHub

Полезные репозитории с реализацией Ford-Fulkerson и максимального потока:
odubno/ford-fulkerson-max-flow

URL: https://github.com/odubno/ford-fulkerson-max-flow

Описание: Python реализация с BFS (Edmonds-Karp)

Язык:

Лицензия: MIT

Algorithms-and-Data-Structures (NetworkX)

URL: https://github.com/networkx/networkx

Описание: Полнофункциональная библиотека для анализа графов

Модули: networkx.algorithms.flow

Языки: Python

cp-algorithms (Competitive Programming)

URL: https://github.com/e-maxx-eng/e-maxx-eng

Описание: Максимальный поток - Форд-Фалкерсон и Edmonds-Karp

URL статьи: https://cp-algorithms.com/graph/edmonds_karp.html

### 6.2 Kaggle

Датасеты и ноутбуки с анализом потоков и сетей:
Network Flow Analysis Notebooks

URL: https://www.kaggle.com/search?q=maximum+flow+algorithm

Тип: Jupyter Notebooks с примерами
Graph Theory and Algorithms

URL: https://www.kaggle.com/datasets (поиск по "graph algorithms")
Pipeline Optimization

URL: https://www.kaggle.com/search?q=pipeline+optimization

### 6.3 Дополнительные ресурсы

Статьи и документация:

GeeksforGeeks: https://www.geeksforgeeks.org/ford-fulkerson-algorithm-for-maximum-flow-problem/

Wikipedia: https://en.wikipedia.org/wiki/Ford–Fulkerson_algorithm

TopCoder: Максимальный поток (максимально подробная статья)

Brilliant.org: https://brilliant.org/wiki/ford-fulkerson-algorithm/

## 7. Выводы

### 7.1 Ключевые результаты

Алгоритм Форда-Фалкерсона эффективно решает задачу максимального потока в сетях
Edmonds-Karp (BFS-версия) гарантирует полиномиальную сложность O(VE2)
Применение к газотранспортной системе позволяет:
Определить максимальные объёмы транспортировки
Найти критические узкие места в сети
Оптимизировать распределение газа между потребителями

### 7.2 Практическое значение

Для российской газотранспортной системы:
Система обслуживает более 150 млрд м³/год экспорта и внутреннего потребления
Алгоритм помогает при:
Планировании развития новых маршрутов
Оптимизации загрузки существующих трубопроводов
Анализе влияния отключений линий на общий поток
Прогнозировании максимальных мощностей

### 7.3 Рекомендации для дальнейшего изучения

Диниц алгоритм — более быстрая реализация (O(V2E))
Push-relabel алгоритм — параллельные вычисления
Минимальный поток с ограничениями — более реалистичные модели
Многоприемниковые сети — несколько потребителей газа одновременно
Динамические потоки — изменение пропускных способностей во времени

## Приложение: Полный код для запуска


In [ ]:
if __name__ == "__main__":
    # Запуск полной демонстрации
    demonstrate_russian_gas_network()
    print("\n" + "="*70)
    print("ПРАКТИКУМ ЗАВЕРШЁН")
    print("="*70)
    print()



ПРАКТИКУМ ЗАВЕРШЁН

